# Clean NC Datasets

In [ ]:
from matplotlib import pyplot as plt
import sys
import os

sys.path.append("..")

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# plt.style.use('ggplot')
plt.rcParams["axes.facecolor"] = "white"

from config import ROOT

os.chdir(ROOT)

In [ ]:
import xarray as xr
import numpy as np

file_path = "data/source/natural/prec_CMFD_V0106_B-01_01yr_010deg_1979-2018.nc"

ds = xr.open_dataset(file_path)
ds
ds.prec

In [ ]:
import xarray as xr
import numpy as np

file_path = "data/source/natural/temp_CMFD_V0106_B-01_01yr_010deg_1979-2018.nc"

ds = xr.open_dataset(file_path)
ds
ds.temp

In [ ]:
import geopandas as gpd
from func.tools import within_province_mask


provinces = gpd.read_file("data/source/natural/省级行政区.shp")
provinces = provinces.to_crs("epsg:4326")
provinces.plot()

In [ ]:
ds_mean = ds.to_array("prec").mean(dim="time")
ax = provinces.plot(alpha=0.2, color="red", zorder=0.5)
ds_mean.plot(ax=ax, zorder=-1)

In [ ]:
prec = within_province_mask(provinces, ds, "prec", longitude="lon", latitude="lat")
prec.head()

In [ ]:
prec_yr = prec * 365 * 24
prec_yr.to_csv("data/processed/prec.csv")

## All data

In [ ]:
from func.tools import show_files
import tqdm

nc_files = show_files(path="data/source/natural/", full_path=True, suffix="nc")
datasets = {}
for file in tqdm.tqdm(nc_files):
    ds = xr.open_dataset(file)
    for var in ds.data_vars.keys():
        dataset = within_province_mask(
            provinces, ds, var, longitude="lon", latitude="lat"
        )
        long_name = ds.get(var).long_name
        if var == "prec":
            dataset = dataset * 365 * 24  # precipitation rate -> precipitation
        dataset.to_csv(f"data/processed/{var}.csv")
        datasets[var] = long_name

datasets.keys()

In [ ]:
datasets